# validator2

This notebook validates materials citations annotation using Darwin core archives downloaded from GGI server. 

### old docs - delete eventually

This notebook controls execution of GGI using **pyautogui**.

It was written to automate downloading XML files for documents from Insects of Guam I and II
as part of a validation process.

This is a fragile hack which probably runs only on my particular setup.

Run in Jupyter lab using the Python 3 kernel. 

The JupyterLab window should display on my laptop, with GGI displaying on the "big screen" (2560 x 1080)

Remember to push to GitHub.

In [1]:
#import pyautogui as pag
import time
#import mysecrets
import pandas as pd
import subprocess
#from bs4 import BeautifulSoup
import re
#import json
from datetime import datetime
import os

In [2]:
def read_dataset_list():
    '''
    Reads ../dataset-list.md and returns a pandas dataframe
    '''
    
    # Read a markdown file, getting the header from the first row and inex from the second column
    # df = pd.read_table('../dataset-list.md', sep="|", header=0, index_col=1, skipinitialspace=True)
    
    df = pd.read_table('../dataset-list.md', sep="|", header=0, skipinitialspace=True)

    # Drop the left-most and right-most null columns 
    
    df = df.dropna(axis=1, how='all')

    # Drop the header underline row
    
    df = df.iloc[1:]  

    # Left-align strings and column headings
    # df = df.style.set_properties(**{'text-align': 'left'})
    # df = df.set_table_styles([dict(selector = 'th', props=[('text-align', 'left')])])

    # Strip whitespace from end of strings
    
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # Strip whitespace from end of column headers
    
    df.columns = df.columns.str.strip()

    # Drop datasets with no title - we don't need to process these
    
    df = df.drop(df[df.title == 'no title'].index)
    return df

# read_dataset_list()

In [3]:
%%time

def update_github():
    command = f'./update_github.sh'
    result = os.system(command)
    assert result==0, f'{command} failed'  

# update_github()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.01 µs


In [4]:
def create_status_report():
        
    summary_list = []
    for uuid in uuids:
        summary = json.load(open(f'{uuid}_summary.json'))
        summary['uuid'] = uuid
        summary_list.append(summary)
    df_summary = pd.DataFrame(summary_list)
    df_merged = pd.merge(df, df_summary, on='uuid')
    df_merged.drop('status', axis=1, inplace=True)
    #df_merged['validation report'] = f'<a href="{df_merged.uuid}.html">validation report</a>'
    summary_html = df_merged.to_html(index=False)
    
    # Edit so that the table is nicely styled with Bulma
    
    summary_html = summary_html.replace('<table border="1" class="dataframe">', '<table class="table">')
    summary_html = summary_html.replace('<th>', '<th class="has-text-centered">')
    summary_html = summary_html.replace('<td>', '<td class="has-text-centered">')
    
    # Link uuid to validation report
    
    for uuid in uuids:
        summary_html = summary_html.replace(uuid, f'<a href="{uuid}.html">{uuid}</a>')
    
    # Highlight rows which need work
    
    for i, r in df_merged.iterrows():
        if (r['bad collectors'] + r['bad locations'] + r['bad dates']) > 0:
            summary_html = nth_repl(summary_html, '<tr', '<tr class="is-selected"', i+2)  
    
    title_html = f'''
        <p class="title is-1">Insects of Guam Datamining Project</p>
        <p class="subtitle is-3">Status report generated by <b>validator2.ipynb</b> at {datetime.utcnow()} UTC</p>
        <p>Highlited documents failed validation. Click on the **uuid** to see the validation report.</p>
        '''

    html = f'''
        <html>
            <header>
                <meta charset="utf-8">
                <meta name="viewport" content="width=device-width, initial-scale=1">
                <title>mat_cit_chk</title>
                <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bulma@0.9.3/css/bulma.min.css">
            </header>
            <body>
                <section class="section">
                    <div class="container">
                        {title_html}
                        {summary_html}
                    </div>
                </section>
            </body>
        </html>        
        '''
    with open('status_report.html', 'w') as f:
        f.write(html) 
        
# create_status_report()

In [5]:
%%time

def validate_dwca(uuid):
    """
    Downloads a Darwin core archive from the GGI server, unzips it.     
    """
    dwca_url = f'http://tb.plazi.org/GgServer/dwca/{uuid}.zip'
    dwca_file = f'{uuid}.zip'

    # download the DwCA into the current working directory, 
    # overwriting any previous DwCA with same uuid

    command = f'wget -O {dwca_file} {dwca_url}'
    result = os.system(command)
    assert result==0, f'{command} failed'

    # unzip the DwCA, overwriting files:
    #   meta.xml, eml.xml, taxa.txt, occurrences.txt, multimedia.txt, description.txt, distribution.txt, 
    #   media.txt, references.txt, vernaculars.txt

    command = f'unzip -o {dwca_file}'
    result = os.system(command)
    assert result==0, f'{command} failed'  

    command = f'unzip -o {dwca_file}'
    result = os.system(command)
    assert result==0, f'{command} failed'  
       
    taxon_df = pd.read_csv('taxa.txt', sep='\t')
    occ_df = pd.read_csv('occurrences.txt', sep='\t') 
    occ_df.eventDate = occ_df.eventDate.astype(str)

    merged_df = taxon_df.merge(right=occ_df, on='taxonID')
    assert merged_df.shape[0]==occ_df.shape[0],'merged_df does not have same number of records as occ_df'
    
    merged_df = merged_df[['canonicalName','country','locality','recordedBy','eventDate']]
    
    date_list = ['1936']
    merged_df['valid_eventDate'] = merged_df['eventDate'].str[:4].isin(date_list)
    
    author_list = ['O. H. Swezey', 'R. L. Usinger & O. H.Swezey']
    merged_df['valid_recordedBy'] = merged_df['recordedBy'].isin(author_list)
    
    locality_list = ['Sumay Road', 'Tarague Beach', 'Machanao', 'Orote Peninsula']
    merged_df['valid_locality'] = merged_df['locality'].isin(author_list)
    
    country_list = ['Guam']
    merged_df['valid_country'] = merged_df['locality'].isin(country_list)

    s = '<table class="table">\n'
    s += '<thead>\n'
    s += '<tr>\n'
    s += '<th>canonicalName</th>\n'
    s += '<th>country</th>\n'
    s += '<th>locality</th>\n'
    s += '<th>recordedBy</th>\n'
    s += '<th>eventDate</th>\n'
    s += '</tr>\n'
    s += '</thead>\n'
    
    for i,r in merged_df.iterrows():
        s += '<tr>\n'
        s += f'<td><i>{r.canonicalName}</i></td>\n'
        
        if r.valid_country:
            s += f'<td>{r.country}</td>\n'
        else:
            s += f'<td class="is-selected">{r.country}</td>\n'
            
        if r.valid_locality:
            s += f'<td>{r.locality}</td>\n'
        else:
            s += f'<td class="is-selected">{r.locality}</td>\n'
                        
        if r.valid_recordedBy:
            s += f'<td>{r.recordedBy}</td>\n'
        else:
            s += f'<td class="is-selected">{r.recordedBy}</td>\n'
            
        if r.valid_eventDate:
            s += f'<td>{r.eventDate}</td>\n'
        else:
            s += f'<td class="is-selected">{r.eventDate}</td>\n'
            
        s += '</tr>\n'
    s += '</table>\n'
    results_html = s
    
    summary_html = '<h1>summary_html coming soon</h1>'
       
    # Write the validation report
    
    timestamp = datetime.utcnow()
    html = f'''
        <html>
            <header>
                <meta charset="utf-8">
                <meta name="viewport" content="width=device-width, initial-scale=1">
                <title>mat_cit_chk</title>
                <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bulma@0.9.3/css/bulma.min.css">
            </header>
            <body>
                <section class="section">
                    <div class="container">
                        {summary_html}
                        {results_html}
                    </div>
                </section>
            </body>
        </html>        
        '''
    with open(f'{uuid}.html', 'w') as f:
        f.write(html)        
            
    return merged_df
            
validate_dwca('FE566D11FFD2FFF5383F9056FFE3FFEF')

CPU times: user 15.5 ms, sys: 313 µs, total: 15.8 ms
Wall time: 505 ms


,canonicalName,country,locality,recordedBy,eventDate,valid_eventDate,valid_recordedBy,valid_locality,valid_country
0,Hylaeus guamensis,NaN,Sumay Road,O. H. Swezey,1936-06-23,True,True,False,False
1,Lithurgus guamensis,NaN,Tarague Beach,O. H. Swezey,1936-05-17,True,True,False,False
2,Lithurgus guamensis,NaN,Machanao,R. L. Usinger & O. H.Swezey,1936-06-30,True,True,False,False
3,Lithurgus guamensis,NaN,Orote Peninsula,O. H. Swezey,1936-09-27,True,True,False,False
4,Lithurgus guamensis,NaN,Yigo,O. H. Swezey,1936-05-19,True,True,False,False
5,Lithurgus guamensis,NaN,Yigo,O. H. Swezey,1936-10-18,True,True,False,False
6,Megachile laticeps,NaN,Tarague Beach,E. H. Bryan,1936-04-16,True,False,False,False
7,Megachile laticeps,NaN,Agana,Rowley,1936-08-13,True,False,False,False
8,Megachile laticeps,NaN,Barrigada,O. H. Swezey,1936-06,True,True,False,False
9,Megachile laticeps,NaN,Piti,O. H. Swezey,1936-05-12,True,True,False,False


In [6]:
%%time

# MAIN

df = read_dataset_list()

# lets process the first 15 datasets

# open_GGI()
# login()

# Validate each DwCA

print('Validating')

uuids = list(df.iloc[0:5].uuid.values)
for uuid in uuids:
    print(uuid)
    get_dwca(uuid)
    validate_dwca()

print('Creating status report')

create_status_report()

print('Updating GitHub')

update_github()
        
print('FINISHED')

Validating
FFF07216FFA41642FFBAFFE7FFCA482A


NameError: name 'get_dwca' is not defined